In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"
cohort = "GSE158237"

# Input paths
in_trait_dir = "../../input/GEO/Obesity"
in_cohort_dir = "../../input/GEO/Obesity/GSE158237"

# Output paths
out_data_file = "../../output/preprocess/Obesity/GSE158237.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/GSE158237.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/GSE158237.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide a corrected version of the code that properly handles the sample characteristics dictionary.

```python
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any, Union

# 1. Determine if gene expression data is available
# Based on the background information, this appears to be gene expression data from colon biopsies
is_gene_available = True  # Gene expression data appears to be available based on study description

# 2. Determine data availability and conversion functions

# 2.1 For the trait (Obesity)
# BMI is a continuous measure that directly relates to obesity
trait_row = 10  # BMI is in row 10

def convert_trait(value):
    """Convert BMI value to obesity classification (binary)"""
    if value is None or value == '/' or pd.isna(value):
        return None
    
    # Extract numeric value after the colon
    if isinstance(value, str) and ':' in value:
        try:
            bmi_str = value.split(':', 1)[1].strip()
            bmi = float(bmi_str)
            # Use standard BMI threshold for obesity (≥30 kg/m² = obese)
            return 1 if bmi >= 30.0 else 0
        except:
            return None
    return None

# 2.2 For age
age_row = 1  # Age is in row 1

def convert_age(value):
    """Convert age value to a numeric value"""
    if value is None or value == '/' or pd.isna(value):
        return None
    
    # Extract numeric value after the colon
    if isinstance(value, str) and ':' in value:
        try:
            age_str = value.split(':', 1)[1].strip()
            return float(age_str)
        except:
            return None
    return None

# 2.3 For gender
gender_row = 2  # Sex is in row 2

def convert_gender(value):
    """Convert sex value to binary (0 for female, 1 for male)"""
    if value is None or value == '/' or pd.isna(value):
        return None
    
    # Extract value after the colon
    if isinstance(value, str) and ':' in value:
        try:
            sex_str = value.split(':', 1)[1].strip()
            # Assuming 1 = male, 2 = female (common coding in medical datasets)
            if sex_str == '1':
                return 1  # Male
            elif sex_str == '2':
                return 0  # Female
            else:
                return None
        except:
            return None
    return None

# 3. Save metadata with initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait_row is not None
if trait_row is not None:
    try:
        # Define the sample characteristics dictionary from the output
        sample_char_dict = {
            0: ['tissue: colon'], 
            1: ['age (years): 52', 'age (years): 71', 'age (years): 66', 'age (years): 53', 'age (years): 57', 'age (years): 70', 'age (years): 62', 'age (years): 58', 'age (years): 60', 'age (years): 68', 'age (years): 69', 'age (years): 44', 'age (years): 73', 'age (years): 55', 'age (years): 48', 'age (years): 56', 'age (years): 65', 'age (years): 64', 'age (years): 67', 'age (years): 59', 'age (years): 50', 'age (years): 39', 'age (years): 51', 'age (years): 46'], 
            2: ['Sex: 1', 'Sex: 2'], 
            3: ['ast (u/l): 9', 'ast (u/l): 22', 'ast (u/l): 16', 'ast (u/l): 15', 'ast (u/l): 18', 'ast (u/l): 20', 'ast (u/l): 33', 'ast (u/l): 12', 'ast (u/l): 23', 'ast (u/l): 17', 'ast (u/l): 32', 'ast (u/l): 7', 'ast (u/l): 14', 'ast (u/l): /', 'ast (u/l): 21', 'ast (u/l): 19', 'ast (u/l): 31', 'ast (u/l): 13', 'ast (u/l): 30', 'ast (u/l): 29', 'ast (u/l): 35'], 
            4: ['alt (u/l): /', 'alt (u/l): 34', 'alt (u/l): 29', 'alt (u/l): 42', 'alt (u/l): 22', 'alt (u/l): 32', 'alt (u/l): 74', 'alt (u/l): 31', 'alt (u/l): 19', 'alt (u/l): 44', 'alt (u/l): 25', 'alt (u/l): 30', 'alt (u/l): 28', 'alt (u/l): 17', 'alt (u/l): 21', 'alt (u/l): 27', 'alt (u/l): 26', 'alt (u/l): 24', 'alt (u/l): 59', 'alt (u/l): 20', 'alt (u/l): 18', 'alt (u/l): 38', 'alt (u/l): 15', 'alt (u/l): 33'], 
            5: ['triglycerides (mg/dl): 61', 'triglycerides (mg/dl): 115', 'triglycerides (mg/dl): 77', 'triglycerides (mg/dl): 108', 'triglycerides (mg/dl): 87', 'triglycerides (mg/dl): 93', 'triglycerides (mg/dl): 304', 'triglycerides (mg/dl): 118', 'triglycerides (mg/dl): 73', 'triglycerides (mg/dl): 69', 'triglycerides (mg/dl): 127', 'triglycerides (mg/dl): 190', 'triglycerides (mg/dl): 76', 'triglycerides (mg/dl): 70', 'triglycerides (mg/dl): 62', 'triglycerides (mg/dl): 50', 'triglycerides (mg/dl): 42', 'triglycerides (mg/dl): 66', 'triglycerides (mg/dl): 52', 'triglycerides (mg/dl): 99', 'triglycerides (mg/dl): 166', 'triglycerides (mg/dl): 124', 'triglycerides (mg/dl): 129', 'triglycerides (mg/dl): 102', 'triglycerides (mg/dl): 325', 'triglycerides (mg/dl): 90', 'triglycerides (mg/dl): 278', 'triglycerides (mg/dl): 180', 'triglycerides (mg/dl): 130', 'triglycerides (mg/dl): 97'], 
            6: ['total cholesterol (mg/dl): 184', 'total cholesterol (mg/dl): 162', 'total cholesterol (mg/dl): 202', 'total cholesterol (mg/dl): 170', 'total cholesterol (mg/dl): /', 'total cholesterol (mg/dl


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide properly formatted code to analyze the GEO dataset and extract clinical features.

```python
# First, let's examine what files are actually available in the cohort directory
import pandas as pd
import numpy as np
import os
import re
import json

# List files in the directory to understand what's available
files_in_dir = os.listdir(in_cohort_dir)
print(f"Files in directory: {files_in_dir}")

# Check for matrix file which would contain gene expression data
# GEO series matrix file typically contains gene expression data
matrix_files = [f for f in files_in_dir if 'matrix' in f.lower() or 'series' in f.lower()]
is_gene_available = len(matrix_files) > 0 and not any(['mirna' in f.lower() or 'methylation' in f.lower() for f in matrix_files])
print(f"Matrix files: {matrix_files}")
print(f"Gene expression data available: {is_gene_available}")

# Load the series matrix file to extract clinical data
clinical_data = None
unique_values = {}

if matrix_files:
    matrix_file_path = os.path.join(in_cohort_dir, matrix_files[0])
    try:
        # In GEO series matrix files, sample characteristics are typically in rows starting with "!Sample_characteristics_ch1"
        with open(matrix_file_path, 'r') as file:
            lines = file.readlines()
        
        # Extract sample characteristic lines
        characteristic_lines = [line.strip() for line in lines if line.startswith('!Sample_characteristics_ch1')]
        
        if characteristic_lines:
            # Parse these lines into a dataframe
            samples = []
            for line in characteristic_lines:
                parts = line.split('\t')
                samples.append(parts[1:])  # Skip the first part which is the header
            
            # Transpose to get characteristics as rows
            clinical_data = pd.DataFrame(samples).T
            print("\nClinical data sample (first 5 rows):")
            print(clinical_data.head())
            
            # Get unique values for each row to identify trait, age, and gender data
            for i, row in clinical_data.iterrows():
                row_values = row.values  # All values in this row
                unique_vals = set(val for val in row_values if pd.notna(val) and val != '')
                if len(unique_vals) > 0:
                    unique_values[i] = list(unique_vals)
                    print(f"Row {i}: {list(unique_vals)[:3]}{'...' if len(unique_vals) > 3 else ''}")
    except Exception as e:
        print(f"Error loading series matrix file: {e}")
        clinical_data = pd.DataFrame()

# Based on the unique values analysis, identify the row indices for trait, age, and gender
# For now, setting to None; we'll update these after analyzing the data
trait_row = None
age_row = None
gender_row = None

# After examining the unique values, update these variables with actual row indices
# Analyze each row to identify trait, age, and gender
for row_idx, vals in unique_values.items():
    # Convert to string for easier text analysis
    vals_str = ' '.join([str(val).lower() for val in vals])
    
    # Look for trait-related terms (like obesity, BMI, weight)
    if any(term in vals_str for term in ['obese', 'obesity', 'bmi', 'body mass index', 'weight']):
        trait_row = row_idx
        print(f"Identified trait row: {row_idx} with values: {vals[:3]}{'...' if len(vals) > 3 else ''}")
    
    # Look for age-related terms
    elif any(term in vals_str for term in ['age', 'years old', 'year old']):
        age_row = row_idx
        print(f"Identified age row: {row_idx} with values: {vals[:3]}{'...' if len(vals) > 3 else ''}")
    
    # Look for gender-related terms
    elif any(term in vals_str for term in ['gender', 'sex', 'male', 'female']):
        gender_row = row_idx
        print(f"Identified gender row: {row_idx} with values: {vals[:3]}{'...' if len(vals) > 3 else ''}")

# Define conversion functions for each variable based on inspection of unique values

def convert_trait(value):
    """Convert obesity-related trait values to binary (0=non-obese, 1=obese)"""
    if pd.isna(value) or value == '':
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to lowercase for case-insensitive comparison
    if isinstance(value, str):
        value_lower = value.lower()
        
        # Obesity-related terms
        if any(term in value_lower for term in ['obese', 'obesity']):
            return 1
        # Non-obese terms
        elif any(term in value_lower for term in ['normal', 'lean', 'control', 'non-obese']):
            return 0
        # BMI-based classification (if BMI values are provided)
        elif 'bmi' in value_lower:
            try:
                bmi_match = re.search(r'(\d+\.?\d*)', value)
                if bmi_match:
                    bmi = float(bmi_match.group(1))
                    return 1 if bmi >= 30 else 0
            except:
                pass
    
    return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if pd.isna(value) or value == '':
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age value
    if isinstance(value, str):
        try:
            # Look for patterns like "X years" or just numbers
            age_match = re.search(r'(\d+\.?\d*)', value)
            if age_match:
                return float(age_match.group(1))
        except:
            pass
    
    # If value is already numeric
    elif isinstance(value, (int, float)):
        return float(value)
    
    return None

def convert_gender(value):
    """Convert gender values to binary (0=female, 1=male)"""
    if pd.isna(value) or value == '':
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to lowercase for case-insensitive comparison
    if isinstance(value, str):
        value_lower = value.lower()
        
        if any(term in value_lower for term in ['female', 'f', 'woman', 'women']):
            return 0
        elif any(term in value_lower for term in ['male', 'm', 'man', 'men']):
            return 1
    
    return None

# Based on the actual inspection results, determine if the trait data is available
is_trait_available = trait_row is not None

# Save metadata to record dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available and clinical data exists
if is_trait_available and clinical_data is not None and not clinical_data.empty:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide a clean implementation for this step:

```python
import os
import json
import pandas as pd
from typing import Callable, Dict, Any, Optional

# 1. First, let's check for and load the sample characteristics data
if os.path.exists(os.path.join(in_cohort_dir, 'sample_characteristics.json')):
    with open(os.path.join(in_cohort_dir, 'sample_characteristics.json'), 'r') as f:
        sample_chars = json.load(f)
else:
    # If file doesn't exist, create empty dictionary
    sample_chars = {}

# Also check for background information
if os.path.exists(os.path.join(in_cohort_dir, 'background.txt')):
    with open(os.path.join(in_cohort_dir, 'background.txt'), 'r') as f:
        background = f.read()
else:
    background = ""

# Print to review the data
print("Sample Characteristics:")
if sample_chars:
    for key, values in sample_chars.items():
        unique_values = set(values)
        print(f"Key {key}: {len(unique_values)} unique values")
        if len(unique_values) <= 10:  # Print all unique values if there are few
            print(f"  Unique values: {unique_values}")
        else:
            # Print a few examples
            print(f"  Sample values: {list(unique_values)[:5]}")

print("\nBackground Information:")
print(background[:1000] + "..." if len(background) > 1000 else background)

# 2. Load the clinical data if available
clinical_data_path = os.path.join(in_cohort_dir, 'clinical_data.csv')
if os.path.exists(clinical_data_path):
    clinical_data = pd.read_csv(clinical_data_path)
    print(f"\nClinical Data Shape: {clinical_data.shape}")
    print(clinical_data.head())
else:
    clinical_data = pd.DataFrame()
    print("\nNo clinical data found.")

# 3. Check for gene expression data (matrix file)
is_gene_available = False
matrix_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('_matrix.csv') or f == 'matrix.csv']
if matrix_files:
    # Check if it's gene expression data by looking at the file
    matrix_path = os.path.join(in_cohort_dir, matrix_files[0])
    try:
        # Read just a few rows to check content
        gene_sample = pd.read_csv(matrix_path, nrows=5)
        print(f"\nMatrix File Preview ({matrix_files[0]}):")
        print(gene_sample.head())
        
        # If many rows (genes) and first column has gene identifiers, likely gene expression data
        if gene_sample.shape[0] > 0 and len(gene_sample.columns) > 1:
            is_gene_available = True
    except Exception as e:
        print(f"Error reading matrix file: {e}")

print(f"\nGene Expression Data Available: {is_gene_available}")

# Now, based on sample characteristics, determine trait, age, and gender availability
trait_row = None
age_row = None
gender_row = None

# Examine sample characteristics to identify relevant rows
if sample_chars:
    for key, values in sample_chars.items():
        unique_values = set(values)
        
        # Look for obesity-related characteristics (weight, BMI, obesity status)
        if len(unique_values) > 1 and any(obesity_term in ' '.join(str(v) for v in unique_values).lower() 
                                          for obesity_term in ['obesity', 'obese', 'bmi', 'body mass', 'weight']):
            trait_row = int(key)
            
        # Look for age information
        if len(unique_values) > 1 and any(age_term in ' '.join(str(v) for v in unique_values).lower() 
                                         for age_term in ['age:', 'year', 'yr', 'age at']):
            age_row = int(key)
            
        # Look for gender/sex information
        if len(unique_values) > 1 and any(gender_term in ' '.join(str(v) for v in unique_values).lower() 
                                         for gender_term in ['gender:', 'sex:', 'male', 'female']):
            gender_row = int(key)

# Define conversion functions
def convert_trait(value):
    """Convert obesity trait value to binary: 1 for obese, 0 for non-obese/control."""
    if value is None:
        return None
    
    # Extract the value part if it contains a colon
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value_lower = str(value).lower()
    
    # Check for obese/obesity indicators
    if any(term in value_lower for term in ['obese', 'obesity']):
        return 1
    # Check for non-obese indicators
    elif any(term in value_lower for term in ['non-obese', 'lean', 'control', 'normal weight']):
        return 0
    # Try to interpret BMI values if present
    elif 'bmi' in value_lower:
        try:
            # Extract numeric part of BMI
            import re
            bmi_value = float(re.search(r'\d+\.?\d*', value_lower).group())
            return 1 if bmi_value >= 30 else 0  # WHO definition: BMI ≥ 30 is obese
        except:
            return None
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric type."""
    if value is None:
        return None
    
    # Extract the value part if it contains a colon
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    try:
        import re
        age_match = re.search(r'(\d+\.?\d*)', str(value))
        if age_match:
            return float(age_match.group(1))
    except:
        pass
    
    return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male."""
    if value is None:
        return None
    
    # Extract the value part if it contains a colon
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value_lower = str(value).lower()
    
    if any(female_term in value_lower for female_term in ['female', 'f', 'woman', 'girl']):
        return 0
    elif any(male_term in value_lower for male_term in ['male', 'm', 'man', 'boy']):
        return 1
    else:
        return None

# Determine if trait data is available
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract and save clinical features if trait data is available
if is_trait_available and not clinical_data.empty:
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the processed clinical data
    print("\nProcessed Clinical Data Preview:")
    print(preview


### Step 5: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 6: Gene Identifier Review

In [ ]:
# The gene identifiers appear to be numeric/alphanumeric values that are likely probe IDs 
# from a microarray platform, not standard human gene symbols.
# Human gene symbols typically follow a specific naming convention (e.g., BRCA1, TP53)
# and are not purely numeric. These identifiers (like '16657436') are most likely
# probe identifiers from an Affymetrix or similar microarray platform.

# Therefore, these identifiers will need to be mapped to human gene symbols
# for biological interpretation and cross-study comparability.

requires_gene_mapping = True


### Step 7: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 8: Gene Identifier Mapping

In [ ]:
# 1. First, let's determine which columns in gene_annotation correspond to gene IDs and symbols

# Examine the gene annotation DataFrame to find the probe ID column and gene symbol column
# From the preview, we can see:
# - 'ID' column contains identifiers that match those in the gene expression data (e.g., '16657436')
# - For gene symbols, we need to look for a column that contains standard human gene symbols

# Check for a column containing gene symbols
gene_symbol_columns = []
for col in gene_annotation.columns:
    # Look at the first 100 values in each column to check for likely gene symbol patterns
    sample_values = gene_annotation[col].dropna().head(100).astype(str).tolist()
    # Gene symbols typically have patterns like uppercase letters with numbers (e.g., BRCA1)
    if any(re.search(r'^[A-Z][A-Z0-9]{1,9}$', str(val)) for val in sample_values):
        gene_symbol_columns.append(col)

print(f"Potential gene symbol columns: {gene_symbol_columns}")

# If we can't find a direct gene symbol column, look for a column with gene accession numbers
gene_accession_columns = []
for col in gene_annotation.columns:
    sample_values = gene_annotation[col].dropna().head(100).astype(str).tolist()
    # Gene accession numbers often have patterns like NM_, NR_, XM_, etc.
    if any(re.search(r'^(NM_|NR_|XM_|XR_)', str(val)) for val in sample_values):
        gene_accession_columns.append(col)

print(f"Potential gene accession columns: {gene_accession_columns}")

# Based on the column analysis, we need to choose the right columns for mapping
# The 'ID' column in gene_annotation matches the indices in gene_data
probe_id_column = 'ID'

# For gene symbols, use 'GB_ACC' as it appears to contain gene accessions
# which can be used to extract potential gene symbols
gene_symbol_column = 'GB_ACC'

# 2. Create a gene mapping DataFrame using these columns
mapping_df = get_gene_mapping(gene_annotation, probe_id_column, gene_symbol_column)
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Sample of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])